In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm


In [2]:
# Set random seed for reproducibility
np.random.seed(42)

# Define number of countries and years
n_countries = 10
n_years = 5

# Create a panel structure
countries = np.repeat([f'Country_{i+1}' for i in range(n_countries)], n_years)
years = np.tile(np.arange(2000, 2000 + n_years), n_countries)

# Generate random features
gdp = np.random.rand(n_countries * n_years) * 100  # GDP as a continuous variable
policy = np.random.choice([0, 1], size=n_countries * n_years)  # Binary policy variable
target = np.random.choice([0, 1], size=n_countries * n_years)  # Binary outcome

# Create the panel dataset
panel_data = pd.DataFrame({
    'country': countries,
    'year': years,
    'gdp': gdp,
    'policy': policy,
    'target': target
})

# Add dummy variables for countries (fixaed effects)
panel_data = pd.get_dummies(panel_data, columns=['year'], drop_first=True)

# Separate features (X) and target (y)
X = panel_data.drop(['target','country'], axis=1)  # Drop 'year' or add it as dummy variables if needed
y = panel_data['target']

# Add a constant term
X = sm.add_constant(X)


In [3]:
print(X.dtypes)
print(y.dtypes)
X = X.astype({col: 'int32' for col in X.select_dtypes('bool').columns})



const        float64
gdp          float64
policy         int32
year_2001       bool
year_2002       bool
year_2003       bool
year_2004       bool
dtype: object
int32


In [4]:
from stargazer.stargazer import Stargazer


model1 = sm.Logit(y, X).fit()

model2 = sm.Logit(y, X).fit()

stargazer = Stargazer([model1, model2])

stargazer.title("Logistic Regression Results")
stargazer.custom_columns(["Model 1", "Model 2"], [1, 1])
stargazer.show_model_numbers(False)

latex_output=stargazer.render_latex()

with open("regression_table.tex", "w") as f:
    f.write(latex_output)

print(stargazer.render_latex())

Optimization terminated successfully.
         Current function value: 0.585687
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.585687
         Iterations 6
\begin{table}[!htbp] \centering
  \caption{Logistic Regression Results}
\begin{tabular}{@{\extracolsep{5pt}}lcc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{2}{c}{\textit{Dependent variable: target}} \
\cr \cline{2-3}
\\[-1.8ex] & \multicolumn{1}{c}{Model 1} & \multicolumn{1}{c}{Model 2}  \\
\hline \\[-1.8ex]
 const & 0.491$^{}$ & 0.491$^{}$ \\
& (1.006) & (1.006) \\
 gdp & 0.001$^{}$ & 0.001$^{}$ \\
& (0.012) & (0.012) \\
 policy & -1.369$^{**}$ & -1.369$^{**}$ \\
& (0.686) & (0.686) \\
 year_2001 & -1.047$^{}$ & -1.047$^{}$ \\
& (1.047) & (1.047) \\
 year_2002 & -0.150$^{}$ & -0.150$^{}$ \\
& (0.966) & (0.966) \\
 year_2003 & -1.244$^{}$ & -1.244$^{}$ \\
& (1.095) & (1.095) \\
 year_2004 & 0.013$^{}$ & 0.013$^{}$ \\
& (0.981) & (0.981) \\
\hline \\[-1.8ex]
 Observations & 50 & 

In [5]:
from statsmodels.iolib.summary2 import summary_col


summary=summary_col([model1],stars=True,float_format='%0.2f',regressor_order=['const','gdp','policy'])

latex_output = summary.as_latex()
print(latex_output)

\begin{table}
\caption{}
\label{}
\begin{center}
\begin{tabular}{ll}
\hline
           & target   \\
\hline
const      & 0.49     \\
           & (1.01)   \\
gdp        & 0.00     \\
           & (0.01)   \\
policy     & -1.37**  \\
           & (0.69)   \\
year\_2001 & -1.05    \\
           & (1.05)   \\
year\_2002 & -0.15    \\
           & (0.97)   \\
year\_2003 & -1.24    \\
           & (1.09)   \\
year\_2004 & 0.01     \\
           & (0.98)   \\
\hline
\end{tabular}
\end{center}
\end{table}
\bigskip
Standard errors in parentheses. \newline 
* p<.1, ** p<.05, ***p<.01
